<a href="https://colab.research.google.com/github/BrendanCreates/Finetuning-Mistral-7B-Legal/blob/Brendan's-Parallel-Branch/ese577_fall25_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BEFORE YOU START, CHANGE THE RUNTIME TO REQUEST A T4 GPU

In [ ]:
# Installs
# You will need at a minimum the following packages. Feel free to install
# additional ones as needed
!pip install google-generativeai
!pip install datasets
!pip install -U bitsandbytes
!pip install transformers
!pip install -U peft
!pip install -U "huggingface_hub[cli]"
!pip install -U trl

In [ ]:
import google.generativeai as genai
from datasets import Dataset, DatasetDict
import pandas as pd
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, \
    BitsAndBytesConfig, TrainingArguments, pipeline, logging
import torch
from trl import SFTTrainer

In [ ]:
# In this block, I include code for parsing the Intro chapter from a text file.
# I ran the following two lines on a linux terminal. You can find the equi-
# valent for your OS or find an online tool for converting PDF into text.
# >>> pdftotext -nopgbrk MIT_6390_chapter_Introduction.pdf
# >>> sed -r ':a /[a-zA-Z,\ ]$/N;s/(.)\n/\1 /;ta' \
#        MIT_6390_chapter_Introduction.txt > \
#        MIT_6390_chapter_Introduction_reformat.txt
#
# Once the PDF was converted to a text file, I manually looked through it to:
# - Remove ninformative lines (e.g., "Last updated: ...", "MIT 6390", ...)
# - Remove comments, which come somewhat poorly organized
# - Remove double line breaks
# - Fix up equations a bit so that they made sense in text format
#
# I did all this in a simple text editor (I used Sublime Text). Then I simply
# uploaded the file to Colab and ran the following code to split the text into
# informative paragraphs. This required a bit of iterating back and forth to
# make sure that no paragraph was "trailing" from the previous one.

import glob

for file in glob.glob("data/source/MIT_6390_chapter_*.txt"):
  with open(file) as f:
    lines = f.readlines()

  paragraphs = []
  min_chars = 200
  par = lines[0]
  for ln in lines[1:]:
    if ln[0] == "•" or ln[0].isdigit() and ln[1:3] == ". ":
      # Part of a list, combine with previous items
      par += ln
    else:
      paragraphs.append(par.strip())  # Remove trailing whitespace and store
      par = ln               # Start new paragraph

  paragraphs.append(par)

for line in paragraphs:
  print(line)
  print('---')
print(len(paragraphs))

# I have uploaded the reformatted text file I used to generate data for the
# Intro chapter. You should follow a similar process to create data from any
# source you wish to use. Note: the better you clean up your data, the more
# useful your final model will be.

In [ ]:
# This block contains the code to interact with the Google Gemini 2.5 Flash API
# to request questions and answers for your data. It loops through a list of
# paragraphs and requests Gemini to create one question for each paragraph indi-
# vidually. Your main job here is to write appropriate prompts that lead Gemini
# to generate useful questions. You can also consider generating more/fewer
# questions per paragraph, or merging paragraphs if you think that will help.
# Be sure to document any changes you make in your report!

# TODO: create your own Gemini API key, and either paste it here (and then
# remove it before turning in your report) or save it in a file and load it here.
geminiApiKey= ...
genai.configure(api_key=geminiApiKey)
cfg = genai.types.GenerationConfig(max_output_tokens=4000)
sys_msg_train = (
'''

  TODO: write your own system prompt that encourages Gemini to generate data
  in the format you want it. Giving examples to Gemini is often useful.

'''
)
print(sys_msg_train)
print()
model_train = genai.GenerativeModel('gemini-2.5-flash', system_instruction=sys_msg_train)
qa_pairs_train = []
for par in paragraphs[:5]:
  qa_pairs_train.append(model_train.generate_content(par, generation_config=cfg).text)
  print(qa_pairs_train[-1])

In [ ]:
sys_msg_val = (

'''

  TODO: write your own system prompt that encourages Gemini to generate data
  in the format you want it for val. Giving examples to Gemini is often useful.

'''

)
print(sys_msg_val)
print()
model_val = genai.GenerativeModel('gemini-2.5-flash', system_instruction=sys_msg_val)
qa_pairs_val = []
for par in paragraphs[:5]:
  qa_pairs_val.append(model_val.generate_content(par, generation_config=cfg).text)
  print(qa_pairs_val[-1])

In [ ]:
# Create HuggingFace datasets
dataset_train = Dataset.from_pandas(pd.DataFrame(qa_pairs_train, columns=["text"]))
dataset_val = Dataset.from_pandas(pd.DataFrame(qa_pairs_val, columns=["text"]))
dataset = DatasetDict({"train": dataset_train, "test": dataset_val})
print(dataset)

In [ ]:
!huggingface-cli login --token <TODO: CREATE YOUR OWN TOKEN>

In [ ]:
# Load the model -- Skeleton
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    '''
      TODO: find an appropriate configuration to use here for quantization
    '''
)

model = AutoModelForCausalLM.from_pretrained(
    '''
      TODO: find an appropriate configuration to use here for finetuning
    '''
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

In [ ]:
# LoRA config -- Skeleton
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    '''
      TODO: find an appropriate configuration to use here for LoRA
    '''
)
model = get_peft_model(model, peft_config)

In [ ]:
# Hyperparameters -- Skeleton
training_arguments = TrainingArguments(
    '''
      TODO: find an appropriate configuration to use here for training
    '''
)

In [ ]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)
trainer.train()

In [ ]:
# Save the model
trainer.model.save_pretrained("ESE577_chatbot")
model.config.use_cache = True
model.eval()

In [ ]:
# Run the model locally
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
def build_prompt(question):
  prompt = f"<s>[INST]@ESE577. {question}. [/INST]"
  return prompt

while True:
  question = input("Enter your ESE577-related question (hit Enter to exit): ").strip()
  if not question:
    break
  prompt = build_prompt(question)
  answer = pipe(prompt)
  print(answer[0]["generated_text"])
  print()
